In [2]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
from modules.constants import AWSConfig, TorchParams

In [3]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_session.region_name
training_data_uri = "s3://{}/".format(AWSConfig.bucket)
metric_definitions = [
        {"Name": "loss", "Regex": "loss: ([0-9\\.]+)"},
        {"Name": "accuracy", "Regex": "accuracy: ([0-9\\.]+)"},
        {"Name": "val_loss", "Regex": "val_loss: ([0-9\\.]+)"},
        {"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"},
    ]

Couldn't call 'get_role' to get Role ARN from role name AmazonSageMaker-ExecutionRole-20220803T052033 to get Role path.
Assuming role was created in SageMaker AWS console, as the name contains `AmazonSageMaker-ExecutionRole`. Defaulting to Role ARN with service-role in path. If this Role ARN is incorrect, please add IAM read permissions to your role or supply the Role Arn directly.


In [9]:
estimator = PyTorch(entry_point='speach_emotion_recognition_vit.py',
                            hyperparameters=TorchParams.hyper_param,
                            role=role,
                            source_dir=".",
                            framework_version='1.6',
                            py_version='py3',
                            instance_count=2,
                            instance_type='ml.c5.xlarge')

In [ ]:
estimator.fit(training_data_uri, logs=True)

2022-08-03 20:59:58 Starting - Starting the training job...
2022-08-03 21:00:22 Starting - Preparing the instances for trainingProfilerReport-1659560398: InProgress
.........
2022-08-03 21:01:42 Downloading - Downloading input data...
2022-08-03 21:02:22 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-08-03 21:02:39,704 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-08-03 21:02:39,707 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-08-03 21:02:39,716 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-08-03 21:02:39,720 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-08-03 21:02:40,029 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requ

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
import numpy as np
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = predictor.predict(images.numpy())
        _, predicted = torch.max(torch.from_numpy(np.array(outputs)), 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
estimator.delete_endpoint()